In [1]:
from binance.client import Client
import pandas as pd
import ta
import numpy as np
import plotly.express as px
import csv
import time

In [2]:
client = Client('Y0o47oI29xuCrJOGaGBvlYhMTSujixeKh3btJszTy35ewGo8xAPSRwLHtwtQB2LC', '8PYmpvMIelPQLAgP5sriWvgGOcBbYZ4vK3Jk4EbznDjL29VpTWytRcpceYkFsErU')

In [3]:
def getdata(symbol, interval, start, end):
    df = pd.DataFrame(client.get_historical_klines(symbol, interval, start, end))
    df = df.iloc[:, :9]
    df.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close_time', 'Quote_av', 'Trades']
    # drop close_time and Quete_av
    df = df.drop(['Close_time', 'Quote_av'], axis=1)
    # convert to datetime
    df['Time'] = pd.to_datetime(df['Time'], unit='ms')
    # set index
    df = df.set_index('Time')
    # convert to float
    df = df.astype(float)
    return df

In [4]:
def indicators(df):
    # calculate 200 SMA
    df['SMA200'] = ta.trend.sma_indicator(df['Close'], window=200)
    # calculate distance between close and 200 SMA in %
    df['SMA200_dist'] = (df['Close'] - df['SMA200']) / df['SMA200'] * 100
    df = df.dropna()
    return df

In [5]:
def getsymbols(random=True, amount=5):
    if random:
        # get 5 random symbols from cryptos.csv
        symbols = pd.read_csv('cryptos.csv')
        symbols = symbols.sample(amount)
        symbols = symbols['Symbol'].tolist()
        return symbols
    if random == False:
        # get all symbols from cryptos.csv
        symbols = pd.read_csv('cryptos.csv')
        symbols = symbols['Symbol'].tolist()
        return symbols

In [83]:
symbols = getsymbols(random=False)
symbols

['ETHUSDT',
 'XRPUSDT',
 'SOLUSDT',
 'MATICUSDT',
 'ETHWUSDT',
 'DOGEUSDT',
 'LINKUSDT',
 'ATOMUSDT',
 'ETCUSDT',
 'ADAUSDT',
 'NEARUSDT',
 'CHZUSDT',
 'APEUSDT',
 'BNBUSDT',
 'ENSUSDT',
 'LINAUSDT',
 'AVAXUSDT',
 'SUSHIUSDT',
 'SANDUSDT',
 'LUNA2USDT',
 'DOTUSDT',
 'UNIUSDT',
 'INJUSDT',
 'RSRUSDT',
 'OPUSDT',
 'LTCUSDT',
 'CRVUSDT',
 'LDOUSDT',
 'SHIB1000USDT',
 'GMTUSDT',
 'AXSUSDT',
 'BCHUSDT',
 'EGLDUSDT',
 'HNTUSDT',
 'EOSUSDT',
 'GALAUSDT',
 'FTMUSDT',
 'LITUSDT',
 'AAVEUSDT',
 'ALGOUSDT',
 'FILUSDT',
 'UNFIUSDT',
 'XLMUSDT',
 'KNCUSDT',
 'RUNEUSDT',
 'GLMRUSDT',
 'COTIUSDT',
 'TRXUSDT',
 'SLPUSDT',
 'MANAUSDT',
 'THETAUSDT',
 'BITUSDT',
 'FITFIUSDT',
 'KAVAUSDT',
 'ICPUSDT',
 'FLMUSDT',
 'RENUSDT',
 'TRBUSDT',
 'DYDXUSDT',
 'MKRUSDT',
 'SNXUSDT',
 'WAVESUSDT',
 'CVXUSDT',
 'MTLUSDT',
 'ZILUSDT',
 'XTZUSDT',
 'LOOKSUSDT',
 'AGLDUSDT',
 'ZECUSDT',
 'HBARUSDT',
 'RVNUSDT',
 'FLOWUSDT',
 'VETUSDT',
 'SUNUSDT',
 'DODOUSDT',
 'GALUSDT',
 'ENJUSDT',
 'XMRUSDT',
 'KDAUSDT',
 'FTTUSDT',

In [84]:
with open('200sma_dist.csv', 'w') as f:
    write = csv.writer(f)
    write.writerow(['Symbol', 'Close', 'SMA200', 'SMA200_dist'])
    for symbol in symbols:
        try:
            data = getdata(symbol, '4h', '2022-09-01', '2022-10-15')
            data = indicators(data)
            # drop open, high, low, volume, trades
            data = data.drop(['Open', 'High', 'Low', 'Volume', 'Trades'], axis=1)
            print(symbol)
            # remove all rows exept the last one
            data = data.iloc[-1:]
            print(data)
            write.writerow([symbol, data['Close'].values[0], data['SMA200'].values[0], data['SMA200_dist'].values[0]])
            print('-------------------------------------------------')
            time.sleep(1)
        except Exception as e:
            print(e)
            print('-------------------------bad symbol', symbol)
            time.sleep(1)
            continue



ETHUSDT
                       Close      SMA200  SMA200_dist
Time                                                 
2022-10-14 20:00:00  1297.25  1374.41345    -5.614282
-------------------------------------------------
XRPUSDT
                      Close    SMA200  SMA200_dist
Time                                              
2022-10-14 20:00:00  0.4872  0.444233     9.672177
-------------------------------------------------
SOLUSDT
                     Close    SMA200  SMA200_dist
Time                                             
2022-10-14 20:00:00  30.09  32.97885    -8.759705
-------------------------------------------------
MATICUSDT
                      Close    SMA200  SMA200_dist
Time                                              
2022-10-14 20:00:00  0.7855  0.796459    -1.375965
-------------------------------------------------
APIError(code=-1121): Invalid symbol.
-------------------------bad symbol ETHWUSDT
DOGEUSDT
                       Close    SMA200  SMA200_dist
Time

In [17]:
def sort_data():
    frame = pd.read_csv('200sma_dist.csv')
    frame.columns = ['Symbol', 'Close', 'SMA200', 'SMA200_dist']
    frame = frame.set_index('Symbol')

    # get 10 highest SMA200_dist values
    highest = frame.sort_values(by='SMA200_dist', ascending=False).head(10)
    print(highest)
    print('---------------------------------')

    # get 10 lowest SMA200_dist values
    lowest = frame.sort_values(by='SMA200_dist', ascending=True).head(10)
    print(lowest)
    print('---------------------------------')

    # get 10 SMA200_dist values closest to 0
    closest = frame.sort_values(by='SMA200_dist', key=lambda x: np.abs(x - 0)).head(10)
    print(closest)
    print('---------------------------------')

In [18]:
sort_data()

                Close      SMA200  SMA200_dist
Symbol                                        
SUNUSDT      0.008300    0.005738    44.653485
ENSUSDT     19.650000   15.440600    27.261894
ACHUSDT      0.013910    0.011771    18.173284
MKRUSDT    900.000000  762.025000    18.106361
SUSHIUSDT    1.371000    1.162650    17.920268
REEFUSDT     0.006061    0.005164    17.367983
DYDXUSDT     1.533000    1.320795    16.066460
JSTUSDT      0.028930    0.025806    12.107450
XRPUSDT      0.487200    0.444233     9.672177
WOOUSDT      0.153100    0.143092     6.993728
---------------------------------
               Close     SMA200  SMA200_dist
Symbol                                      
KNCUSDT     0.927000   1.318850   -29.711491
JASMYUSDT   0.005106   0.006975   -26.791868
KLAYUSDT    0.143500   0.195144   -26.464748
RVNUSDT     0.030500   0.040815   -25.272571
OPUSDT      0.713000   0.937915   -23.980318
NEARUSDT    2.980000   3.772540   -21.008127
LDOUSDT     1.296000   1.626245   -20.3072